In [127]:
import itertools
import operator
import sys
import os
import numpy as np
from tqdm.notebook import tqdm
sys.path.append("../src")
from glob import glob
import pandas as pd
import json
from matplotlib import pyplot as plt

from neuraldb.scoring.r_precision import f1, recall


In [132]:
search_root = "/checkpoint/jth/job_staging/neuraldb_expts/experiment=ssg_final"
checkpoint_name = "metrics_test.json"
files = glob("{}*/**/{}".format(search_root,checkpoint_name), recursive=True)

print(len(files))

27


In [133]:

def expand(idx,chunk):
  #elif idx == 1:
  #  return ["experiment={}".format(chunk)]
  if chunk.startswith("seed-"):
    return ["seed={}".format(chunk.replace("seed-",""))]
  elif "," in chunk:
    return chunk.split(",")
  elif "=" in chunk:
    return [chunk]

  return []

experiments = []
for file in files:
    chunks = file.split("/")
    chunks = itertools.chain(*[expand(idx, chunk) for idx, chunk in enumerate(chunks)])

    data = {k:v for k,v in (chunk.split("=") for chunk in chunks)}
    data["file"] = file
    data['dir'] = os.path.dirname(file)
    experiments.append(data)


print(len(experiments))

27


In [146]:
from collections import defaultdict


for experiment in tqdm(experiments):
    all_raw = []
    em = 0.0
    with open(experiment['file']) as f:
        for line in f:
            partial_results = json.loads(line)
            all_raw.extend(partial_results['test']['raw'])

    experiment["EM"] = np.mean([rec[2] for rec in all_raw])
    experiment["raw"] = all_raw

    gold = defaultdict(lambda: defaultdict(list))
    for instance in experiment["raw"]:
        query_type = instance[3]["instance"]["metadata"]["query_type"]
        question = instance[3]["instance"]["question"]

        #if instance[3]["instance"]["metadata"]["query_type"] in  {"min/max", "count", "set"}:
        question = question+"_{}".format(instance[3]['instance']['db_id'])
        gold[query_type][question].append((instance[0], instance[1]))


    aem = 0
    aem_count = 0

    scores = defaultdict(int)
    counts = defaultdict(int)

    count_acts = []
    count_preds = []
    for t, questions in gold.items():

        if t in {"atomic_boolean","join_boolean", "atomic_extractive","join_extractive"}:
            for question, answers in questions.items():
                correct = set()
                predicted = set()

                for answer in answers:
                    if answer[0] != "[NULL_ANSWER]":
                        predicted.add(answer[0])

                    if answer[1] != "[NULL_ANSWER]":
                        correct.add(answer[1])

                aem_count+=1
                #counts[t]+=1

                aem +=1.0 if f1(correct,predicted) == 1.0 else 0
                #scores[t] += 1.0 if f1(correct,predicted) == 1.0 else 0


                scores[t+"_recall"] += recall(correct,predicted)
                scores[t+"_f1"] += f1(correct,predicted)
                scores[t+"_em"] += 1.0 if f1(correct,predicted) == 1.0 else 0

                counts[t+"_recall"]+=1
                counts[t+"_f1"]+=1
                counts[t+"_em"]+=1

                if "atomic" in t:
                    counts["all_atomic"] += 1
                    scores["all_atomic"] += 1.0 if f1(correct,predicted) == 1.0 else 0
                elif "join" in t:
                    counts["all_join"] += 1
                    scores["all_join"] += 1.0 if f1(correct,predicted) == 1.0 else 0

                    # if answer[0] == "[NULL_ANSWER]":
                    #     continue
                    # aem_count +=1
                    # counts[t] += 1
                    #
                    # if answer[0] == answer[1]:
                    #     aem += 1
                    #     scores[t]+=1
        elif t == "min/max":

            for question, answers in questions.items():

                argmin_aggr_gold = defaultdict(list)
                argmin_aggr_pred = defaultdict(list)
                for answer in answers:
                    if answer[0] != "[NULL_ANSWER]" and "[LIST]" in answer[0]:
                        key,value = answer[0].split("[LIST]",maxsplit=1)
                        argmin_aggr_pred[key.strip()] = value.strip()

                    if answer[1] != "[NULL_ANSWER]":
                        key,value = answer[1].split("[LIST]",maxsplit=1)
                        argmin_aggr_gold[key.strip()] = value.strip()

                min_item_gold = sorted(argmin_aggr_gold.items(),key=lambda item: item[1])
                min_item_pred = sorted(argmin_aggr_pred.items(),key=lambda item: item[1])

                max_item_gold = sorted(argmin_aggr_gold.items(),key=lambda item: item[1],reverse=True)
                max_item_pred = sorted(argmin_aggr_pred.items(),key=lambda item: item[1],reverse=True)

                aem_count +=1
                counts[t] +=1
                if len(min_item_pred) and len(max_item_pred):
                    if min_item_gold[0][0] == min_item_pred[0][0] or max_item_gold[0][0] == max_item_pred[0][0]:
                        aem+=1
                        scores[t]+=1


        elif t == "set":

            for question, answers in questions.items():
                set_gold = set()
                set_pred = set()
                for answer in answers:
                    if answer[0] != "[NULL_ANSWER]":
                        set_pred.add(answer[0].strip())

                    if answer[1] != "[NULL_ANSWER]":
                        set_gold.add(answer[1].strip())

                aem_count +=1
                counts[t] +=1
                aem += f1(set_gold, set_pred)
                scores[t] += f1(set_gold, set_pred)


        elif t == "count":
            for question, answers in questions.items():

                set_gold = set()
                set_pred = set()
                for answer in answers:
                    if answer[0] != "[NULL_ANSWER]" and answer[0] != "0":
                        set_pred.add(answer[0].strip())

                    if answer[1] != "[NULL_ANSWER]" and answer[1] != "0":
                        set_gold.add(answer[1].strip())


                aem_count +=1
                aem += 1 if len(set_gold) == len(set_pred) else 0
                scores[t] += 1 if len(set_gold) == len(set_pred) else 0
                counts[t] += 1

                count_acts.append(len(set_gold))
                count_preds.append(len(set_pred))


    ape = sum([abs(act-pred)/act if act else 0 for act,pred in zip(count_acts,count_preds)])/len(count_preds)*100
    aae = sum([abs(act-pred) if act else 0 for act,pred in zip(count_acts,count_preds)])/len(count_preds)
    experiment["A_count_ape"] = ape
    experiment["A_count_aae"] = aae
    print(ape,aae)
    for k,v in counts.items():
        experiment["A_type_{}".format(k)] = scores[k]/v

    experiment["A_EM"] =aem/aem_count

9.34199503253212 0.2557544757033248
9.032755394647985 0.26342710997442453
9.934594391116141 0.2672634271099744
8.005957105778027 0.4106217616580311
8.582860427820876 0.4209844559585492
8.396687534131026 0.44041450777202074
8.67185741226662 0.23657289002557544
8.71863962976495 0.2391304347826087
9.229138295634463 0.2647058823529412
9.774904691784494 0.2710997442455243
9.482812228975918 0.2531969309462916
9.952761078080774 0.2672634271099744
9.722628181707474 0.2557544757033248
9.676251923694384 0.2608695652173913
9.789007170720733 0.26214833759590794
8.90842353097514 0.37176165803108807
7.063453962017488 0.36658031088082904
8.125953774131526 0.3484455958549223
7.375270225997397 0.3963730569948187
7.3354463271299934 0.34974093264248707
7.6611592817637435 0.41580310880829013
10.22284512705969 0.27458492975734355
8.303335305251016 0.23499361430395913
8.681814495990746 0.24265644955300128
47.884210652409855 0.859514687100894
110.79142086804924 1.9118773946360152
56.15459466034176 0.97956577

In [138]:
results = pd.DataFrame(experiments).fillna(0)

cols = {col:[np.mean, np.std] for col in filter(lambda col: col == "A_EM" or col == "EM" or col.startswith("A_") or col.startswith("prop_") or col.startswith("type_"),results.columns)}
cols.update({col:[np.max] for col in filter(lambda col: col.startswith("count_type_negative"), results.columns)})
breakdown_cols = list(filter(lambda col: col.startswith("prop_"),results.columns))
type_cols = list(filter(lambda col: (col.startswith("type_") and "negative" not in col) or col.startswith("x"),results.columns))
print(type_cols)
type_cols2 = list(filter(lambda col: col.startswith("type_") and "negative" not in col or col == "x_avg_negative",results.columns))
a_type_cols = list(filter(lambda col: col.startswith("A_type_") and "negative" not in col or col == "x_avg_negative",results.columns))
type_cols3 = list(filter(lambda col: "count" not in col and  "negative" in col,results.columns))
type_cols4 = list(filter(lambda col: col.startswith("type_") and "negative" not in col,results.columns))
breakdown = pd.pivot_table(results, index=["experiment","model","lr", "method"],columns=[],aggfunc=cols)
#pd.option_context("display.max_rows",None)
pd.options.display.max_rows = 150
pd.options.display.max_columns = 150
breakdown


[]


A_EM           A_count_ape  \
                                            mean       std        mean   
experiment model   lr   method                                           
ssg_final  t5-base 4e-4 both            0.868217  0.004944   24.936061   
                        ds              0.867029  0.001569   26.342711   
                        duo_both        0.824720  0.015811  129.118774   
                        duo_ds          0.934104  0.003107   25.117071   
                        duo_supervised  0.945773  0.001063   38.816926   
                        fix_both        0.910832  0.006640   26.555840   
                        fix_ds          0.913015  0.000879   26.172208   
                        fix_supervised  0.942161  0.004047   36.658031   
                        supervised      0.896910  0.001814   42.530225   

                                                  A_type_all_atomic            \
                                              std              mean       std   
experiment model   lr   method                                                  
ssg_final  t5-base 4e-4 both             1.672212          0.890840  0.009040   
                        ds               0.586007          0.892390  0.002329   
                        duo_both        58.540919          0.957062  0.008304   
                        duo_ds           2.076415          0.984141  0.000686   
                        duo_supervised   3.341199          0.985955  0.002469   
                        fix_both         0.993280          0.975723  0.006284   
                        fix_ds           0.295320          0.978994  0.002088   
                        fix_supervised   1.187196          0.987268  0.003232   
                        supervised       1.512459          0.927289  0.001218   

                                       A_type_all_join            \
                                                  mean       std   
experiment model   lr   method                                     
ssg_final  t5-base 4e-4 both                  0.691304  0.027152   
                        ds                    0.685507  0.013283   
                        duo_both              0.866667  0.022222   
                        duo_ds                0.910053  0.006608   
                        duo_supervised        0.971609  0.003155   
                        fix_both              0.700000  0.041929   
                        fix_ds                0.702899  0.015269   
                        fix_supervised        0.969506  0.001821   
                        supervised            0.806519  0.001821   

                                       A_type_atomic_boolean_em            \
                                                           mean       std   
experiment model   lr   method                                              
ssg_final  t5-base 4e-4 both                           0.909950  0.011202   
                        ds                             0.914925  0.003253   
                        duo_both                       0.971166  0.001840   
                        duo_ds                         0.988548  0.000937   
                        duo_supervised                 0.988569  0.001968   
                        fix_both                       0.980348  0.004856   
                        fix_ds                         0.983831  0.001140   
                        fix_supervised                 0.991342  0.001636   
                        supervised                     0.966811  0.001889   

                                       A_type_atomic_boolean_f1            \
                                                           mean       std   
experiment model   lr   method                                              
ssg_final  t5-base 4e-4 both                           0.962687  0.004422   
                        ds                             0.964179  0.000746   
                        duo_both                       